<a href="https://colab.research.google.com/github/albertbolanoss/labs_deep_learning/blob/download_dataset/notebooks/simpsons-image-classification/01_download_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Simpsons Image Classification (Download dataset)